In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import savgol_filter

In [7]:
# Baca file CSV
file_path = '/run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/Buoys/0N90E/ASCII/convert/rad0n90e_dy.csv'  # Sesuaikan dengan nama file Anda
df = pd.read_csv(file_path)
print(f"Memproses data untuk file {file_path}...")

Memproses data untuk file /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/Buoys/0N90E/ASCII/convert/rad0n90e_dy.csv...


In [8]:
# Konversi timestamp dan set sebagai indeks
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)

In [ ]:
# Hapus baris dengan Q=0 (data tidak berkualitas)
df_quality = df[df['Q'] > 0].copy()

In [ ]:
# 1. Analisis Deskriptif
print("=== Statistik Deskriptif Curah Hujan ===")
print(df_quality['Prec'].describe())


In [ ]:
# 2. Identifikasi dan visualisasi missing values
plt.figure(figsize=(14, 6))
plt.plot(df_quality.index, df_quality['Prec'], 'o-')
plt.title('Curah Hujan di 0N 90E (Data Berkualitas)')
plt.ylabel('Curah Hujan (mm/jam)')
plt.xlabel('Tanggal')
plt.grid(True)
missing_periods = df[df['Q'] == 0].index
for date in missing_periods:
    plt.axvline(x=date, color='r', linestyle='--', alpha=0.3)
plt.savefig('output/curah_hujan_0N90E.png')
plt.close()


In [ ]:
# 3. Analisis Pola Musiman
# Resampling ke data bulanan (menggunakan nilai rata-rata jika ada lebih dari 1 pengukuran per bulan)
monthly_data = df_quality['Prec'].resample('M').mean()

# Membuat kolom bulan dan tahun
monthly_df = pd.DataFrame(monthly_data)
monthly_df['month'] = monthly_df.index.month
monthly_df['year'] = monthly_df.index.year

In [ ]:
# Plot rata-rata curah hujan per bulan (pola musiman)
monthly_pattern = monthly_df.groupby('month')['Prec'].mean()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
plt.figure(figsize=(12, 6))
monthly_pattern.plot(kind='bar')
plt.title('Rata-rata Curah Hujan Bulanan di 0N 90E (2005-2020)')
plt.ylabel('Curah Hujan (mm/jam)')
plt.xlabel('Bulan')
plt.xticks(np.arange(12), months, rotation=45)
plt.grid(True, axis='y')
plt.savefig('output/pola_musiman_0N90E.png')
plt.close()

In [ ]:
# 4. Analisis Tren Jangka Panjang
# Resampling ke data tahunan
annual_data = df_quality['Prec'].resample('YE').mean()


In [ ]:
# Plot tren tahunan
plt.figure(figsize=(14, 6))
annual_data.plot()
plt.title('Tren Tahunan Curah Hujan di 0N 90E (2005-2020)')
plt.ylabel('Rata-rata Curah Hujan (mm/jam)')
plt.xlabel('Tahun')
plt.savefig('output/tren_tahunan_curah_hujan_0N90E.png')
plt.grid(True)

In [ ]:
# Tambahkan garis tren linear
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(len(annual_data)), annual_data)
plt.plot(annual_data.index, intercept + slope * np.arange(len(annual_data)), 'r--', 
         label=f'Tren: {slope:.4f} mm/tahun (p-value: {p_value:.4f})')
plt.legend()
plt.savefig('output/tren_tahunan_0N90E.png')
plt.close()

In [ ]:
# 5. Korelasi antara Curah Hujan dan Persentase Waktu
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Prec', y='%Time', data=df_quality, hue='S')
plt.title('Korelasi antara Curah Hujan dan Persentase Waktu')
plt.xlabel('Curah Hujan (mm/jam)')
plt.ylabel('Persentase Waktu (%)')
plt.grid(True)
plt.savefig('output/korelasi_curah_hujan_waktu_0N90E.png')
plt.close()

In [ ]:
# Setelah semua operasi preprocessing
# Konversi kolom Q dan S menjadi integer
df['Q'] = df['Q'].astype('Int64')  # Perhatikan 'I' kapital
df['S'] = df['S'].astype(int)

In [ ]:
# 6. Export data clean untuk analisis lebih lanjut
df_quality.to_csv('cleaned/r0n90e_clean.csv')

print("Analisis Selesai! Plot disimpan sebagai file PNG")